# Inference Notebook Template

Country Level 3 (L3) means-

* With Continent as cue
* With Explicit Country list

In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"

🔨 TO BE MODIFIED 🔨

In [2]:
# TO BE MODIFIED


!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-a2q73jsu
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-a2q73jsu
  Resolved https://github.com/huggingface/transformers.git to commit 76fc50a1527a7db593a6057903b749598f7000a9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.0/502.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11349228 sha256=3cf69a9212a04c3906a675725d8bef74c816fb565eba56212e5c89a6f069806b
  Stored in directory: /tmp/pip-ephem-wheel-cache-yshm8kok/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Foun

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 📚 Helper: Save any results dict to JSON

In [17]:
import json, os


def save_results(data: dict,
                 model_name: str,
                 variant: str,
                 task: str,
                 task_level: str,
                 prompt_level: str,
                 run_count: str,
                 output_dir: str = "/content/drive/MyDrive/RQ1_Experiments/results"):
    """
    Writes one JSON per prompt level (resilient for long runs).
    Includes task_level so L2/L3 files never collide.
    """
    model_dir = os.path.join(output_dir, model_name)
    os.makedirs(model_dir, exist_ok=True)
    fname    = f"RQ1_{model_name}_{variant}_{task}_{task_level}_{prompt_level}_{run_count}.json"
    out_path = os.path.join(model_dir, fname)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f" Saved results to {out_path}")



🔨 TO BE MODIFIED 🔨

### **Config**

In [18]:
MODEL_NAME   = 'InternVL3-8B'
VARIANT      = '8B'
RUN_COUNT    = 'r1'
PROMPT_LEVEL = 'prompt1'        # will be overridden in the loop per prompt


folder_path  = "/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1"

TASK = 'country'
TASK_LEVEL = 'L3'

# output root in Drive
OUTPUT_DIR = "/content/drive/MyDrive/RQ1_Experiments/results"

# Generation settings
GEN_TEMPERATURE     = 0.3
GEN_MAX_NEW_TOKENS  = 1024
GEN_DO_SAMPLE       = False

# InternVL3 vision tiling
IMAGE_SIZE     = 448
MAX_NUM_TILES  = 4
FALLBACK_TILES = 3

# Quantization: use "none" first. If OOM on load, set to "bnb-8bit".
QUANTIZATION = "none"   # "none" | "bnb-8bit"


### **Dataset discovery**

In [19]:
import os

def get_all_file_paths(root_dir):
    file_paths = []
    stack = [root_dir]
    while stack:
        current_dir = stack.pop()
        with os.scandir(current_dir) as it:
            for entry in it:
                if entry.is_file() and entry.name.lower().endswith((".jpg",".jpeg",".png",".webp",".bmp")):
                    file_paths.append(entry.path)
                elif entry.is_dir():
                    stack.append(entry.path)
    return sorted(file_paths)

assert os.path.exists(folder_path), f"folder_path does not exist: {folder_path}"
image_paths = get_all_file_paths(folder_path)
print(f"Found {len(image_paths)} files.")


Found 360 files.


In [20]:
print(image_paths)

['/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B1_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B3_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B4_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B7_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B8_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B9_MIDDLE.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Cape Verde/Africa_Cape Verde_CV1_LAST.PNG', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Cape Verde/Africa_Cape Verde_CV2_LAST.png', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Cape Verde/Africa_Cape Verde_CV3_LAST.JPG', '/content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Cape Verde/Africa_Cape Verd

# Variant: InternVL3 8B

https://huggingface.co/OpenGVLab/InternVL3-8B

## 1️⃣ Prompt

🔨 TO BE MODIFIED 🔨

In [21]:
# L3 PROMPTS
PROMPTS = {
    "prompt1": """Assume the image is from {given_continent}. Analyze it and answer two things:

1) Which country is this most likely from? (choose from {country_list})

2) What is the most suitable label: political expression, vandalism, decoration, cultural identity, or advertisement?""",

    "prompt2": """You are a cultural anthropologist viewing street visuals. Given that mural in image is located in {given_continent}, give these four pieces of info:

- COUNTRY (choose exactly one: {country_list})
- LABEL (choose exactly one: political expression / vandalism / decoration / cultural identity / advertisement)
- CONFIDENCE (0–100%)
- TOP CUE (2–3 sentences describing the key visual/cultural cue)

Be concise and calibrate CONFIDENCE to how strongly the visual cues support your claims.""",

    "prompt3": """Moderate a brief debate among a Geographer, a Street Artist, and a Museum Curator about this image, assuming it originates from {given_continent}. For each expert, give:

- ROLE: <role name>
- COUNTRY (choose exactly one: {country_list})
- LABEL (political expression / vandalism / decoration / cultural identity / advertisement)
- SHORT_RATIONALE (one concise sentence)

After the three positions, write a 2-paragraph moderator resolution:
1) Compare the three rationales and state which COUNTRY and LABEL you choose and why (trade-offs).
2) List 2 visual cues that were decisive and mention one alternative explanation you rejected.

Finish with: COUNTRY and LABEL (each on its own line).""",

    "prompt4": """You are a cultural anthropologist & artist. Given that mural in image is located in {given_continent}, provide:

- COUNTRY: two most likely countries (choose from: {country_list}) with percentages (sum ≈100), e.g. ""Iceland: 70%, Russia: 30%""
- WHY COUNTRY: 2–3 sentences explaining how visual cues suggest the chosen countries .

- LABEL: choose from (political expression, vandalism, decoration, cultural identity, advertisement) and also assign probabilities for all five labels (sum ≈100)
- WHY LABEL: 2–3 sentences linking cues to chosen label.

- TOP CUES: list three concrete cues that were decisive (1–6 words each)
- BIAS NOTE: 2–3 sentences reflecting how cultural bias might have affected the probability estimates.

Constraints:
- Avoid equal continent splits (e.g. 50/50). Show one as more likely.
- Keep WHY COUNTRY and WHY LABEL tied to visible evidence (colors, symbols, technique, location clues etc)."""
}
PROMPT_ORDER = ["prompt1", "prompt2", "prompt3", "prompt4"]


## 2️⃣ Load Processor and Model

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

🔨 TO BE MODIFIED 🔨

In [22]:
# TO BE MODIFIED

from transformers import AutoModel, AutoTokenizer, AutoProcessor
import torch, os
from PIL import Image
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

repo_id = "OpenGVLab/InternVL3-8B"

model = AutoModel.from_pretrained(
    repo_id,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True
).eval().cuda()

processor = AutoProcessor.from_pretrained(
    repo_id,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(repo_id, trust_remote_code=True, use_fast=True)



Unrecognized keys in `rope_parameters` for 'rope_type'='dynamic': {'rope_theta'}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unrecognized keys in `rope_parameters` for 'rope_type'='dynamic': {'rope_theta'}


## 3️⃣ Inference

In [ ]:
import tempfile

🔨 TO BE MODIFIED 🔨

In [23]:
import numpy as np
import tempfile
from PIL import Image

def infer_img(image_path: str, prompt: str):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    # Build question with the correct image token
    question = prompt if "<image>" in prompt else "<image>\n" + prompt

    # Read & minimally preprocess to a single 448x448 tile
    img = Image.open(image_path).convert("RGB").resize((448, 448))
    arr = np.array(img, dtype=np.float32) / 255.0
    # ImageNet normalize inline
    arr[..., 0] = (arr[..., 0] - 0.485) / 0.229
    arr[..., 1] = (arr[..., 1] - 0.456) / 0.224
    arr[..., 2] = (arr[..., 2] - 0.406) / 0.225
    pixel_values = torch.from_numpy(arr).permute(2, 0, 1).unsqueeze(0)  # [1,3,448,448]
    pixel_values = pixel_values.to(torch.bfloat16)
    if any(p.is_cuda for p in model.parameters()):
        pixel_values = pixel_values.cuda()

    # Generation config
    gen_cfg = dict(
        max_new_tokens=GEN_MAX_NEW_TOKENS,
        do_sample=GEN_DO_SAMPLE,
        temperature=GEN_TEMPERATURE
    )

    # InternVL3 chat API
    with torch.inference_mode():
        response = model.chat(
            tokenizer,
            pixel_values,
            question,
            gen_cfg,
            history=None,
            return_history=False
        )

    return response.strip()


## 4️⃣ Package & Save to JSON

In [15]:
import os, re
from copy import deepcopy

def extract_location_from_path(path):
    """
    From any path where the filename encodes location, e.g.:
      .../dir/Continent_Country_filename.ext
      .../dir/Continent_Country_City_filename.ext
    Returns (continent, country, city_or_None, continent_dir)
    """
    if not path:
        raise ValueError("Empty path provided")

    basename = os.path.basename(path)
    stem, _ext = os.path.splitext(basename)

    tokens = stem.split('_')

    if len(tokens) < 2:
        raise ValueError(f"Filename does not contain continent and country tokens: {basename!r}")

    continent = tokens[0]
    country   = tokens[1]
    city = None

    tier_re = re.compile(r'^(?:tier|t)?\d+$', re.IGNORECASE)
    filename_marker_re = re.compile(r'^(?:img|image|photo|scan|picture|pic)\d*$', re.IGNORECASE)

    if len(tokens) >= 4:
        cand = tokens[2]
        if (re.search(r'[A-Za-z]', cand)
            and not re.search(r'\d', cand)
            and not tier_re.match(cand)
            and not filename_marker_re.match(cand)):
            city = cand
    elif len(tokens) == 3:
        cand = tokens[2]
        if (re.search(r'[A-Za-z]', cand)
            and not re.search(r'\d', cand)
            and not tier_re.match(cand)
            and not filename_marker_re.match(cand)):
            city = cand

    # find a parent dir literally named as the continent (best effort)
    continent_dir = None
    abs_parent = os.path.abspath(os.path.dirname(path))
    cur = abs_parent
    while True:
        if os.path.basename(cur) == continent:
            continent_dir = cur
            break
        parent = os.path.dirname(cur)
        if parent == cur:
            break
        cur = parent
    if continent_dir is None:
        fallback = os.path.join(abs_parent, continent)
        continent_dir = "/" + fallback.lstrip(os.sep)

    return continent, country, city, continent_dir


# Data: 9 countries per continent (TOP(3), MIDDLE(3), LAST(3))
CONTINENT_COUNTRIES = {
    "Africa": [
        "Nigeria", "South Africa", "Egypt",    # TOP (3)
        "Seychelles", "Comoros", "Cape Verde", # MIDDLE (3)
        "Mozambique", "Rwanda", "Botswana"     # LAST (3)
    ],
    "Asia": [
        "China", "India", "Indonesia",
        "United Arab Emirates", "Singapore", "Israel",
        "Timor-Leste", "Maldives", "Brunei"
    ],
    "Europe": [
        "Russia", "Germany", "United Kingdom",
        "Denmark", "Lithuania", "Slovakia",
        "Monaco", "Gibraltar", "San Marino"
    ],
    "North America": [
        "United States", "Mexico", "Canada",
        "Saint Lucia", "Guyana", "Bahamas",
        "British Virgin Islands", "Anguilla", "Montserrat"
    ],
    "South America": [
        "Brazil", "Argentina", "Colombia",
        "Venezuela", "Chile", "Ecuador",
        "Paraguay", "Uruguay", "Falkland Islands"
    ],
    # Added so get_countries never raises for these
    "Oceania": [
        "Australia", "New Zealand", "Papua New Guinea",
        "Fiji", "Samoa", "Tonga",
        "Vanuatu", "Solomon Islands", "Kiribati"
    ],
    "Antarctica": [
        # intentionally empty — no sovereign states
    ],
}

# Normalization helpers
def _norm(s: str) -> str:
    """
    Minimal normalizer used for matching:
      - lowercases and removes non-alphanumeric characters.
    Use this to compare filename tokens to canonical country names.
    """
    return re.sub(r'[^a-z0-9]', '', (s or '').lower())

def get_countries(continent: str, image_path: str = None):
    """
    Return the continent's country list (copy) and optionally apply filename-based replacement.

    Behavior:
      - continent: case-insensitive continent name (e.g. "Africa").
      - image_path: optional filename like "Continent_Country_..._[TOP|MIDDLE|LAST].ext".
        * The country token is taken from the SECOND underscore-separated token (parts[1]).
        * If that country is NOT already in the continent list, the function replaces the
          country at the group index with the parsed country:
            TOP    -> index 2
            MIDDLE -> index 5
            LAST   -> index 8
          (If tag is missing the function uses index 5 by current logic.)
    Returns:
      (countries_list_copy, info_dict)
        - countries_list_copy: a shallow copy of the continent list with any replacement applied.
        - info_dict: { used_continent, filename_country, tag, matched, added, removed, index }
    Raises:
      ValueError: if the provided continent is not recognized.
    """
    cont_key = next((k for k in CONTINENT_COUNTRIES if _norm(k) == _norm(continent)), None)
    if not cont_key:
        raise ValueError(f"Unknown continent: {continent!r}")

    arr = deepcopy(CONTINENT_COUNTRIES[cont_key])
    info = {"used_continent": cont_key, "filename_country": None, "tag": None,
            "matched": None, "added": None, "removed": None, "index": None}

    if not image_path:
        return arr, info

    base = os.path.basename(image_path)
    name, _ = os.path.splitext(base)
    parts = name.split('_')
    if len(parts) < 2:
        return arr, info  # can't parse a country token

    # country is the second token (parts[1]) per filename convention
    country_token = parts[1]
    # detect tag if last token is TOP/MIDDLE/LAST
    tag = parts[-1].upper() if parts[-1].upper() in ("TOP", "MIDDLE", "LAST") else None

    # build display country from single token: replace dashes with space and title-case
    display_country = country_token.replace('-', ' ').title()
    info["filename_country"] = display_country
    info["tag"] = tag

    # check if already present (using normalization)
    for c in arr:
        if _norm(c) == _norm(display_country):
            info["matched"] = c
            return arr, info  # already exists — no change

    # Not present: replace at group index
    tag_to_idx = {"TOP": 2, "MIDDLE": 5, "LAST": 8}
    remove_idx = tag_to_idx.get(tag, 5)
    remove_idx = max(0, min(remove_idx, len(arr) - 1))

    removed = arr.pop(remove_idx)
    arr.insert(remove_idx, display_country)

    info["added"] = display_country
    info["removed"] = removed
    info["index"] = remove_idx
    return arr, info


In [ ]:
import re, json, os, torch, tempfile
import numpy as np
from PIL import Image

def strip_code_fence(s: str) -> str:
    """
    Remove leading/trailing triple-backtick fences (and any 'json' marker)
    and trim whitespace.
    """
    if not isinstance(s, str):
        return s
    s = s.strip()
    s = re.sub(r"^```(?:\s*json)?\s*", "", s, flags=re.I)
    s = re.sub(r"```$", "", s)
    return s.strip()

def try_parse_json_from_string(s: str):
    """
    Try to parse JSON from a string. Returns (parsed_obj, error_message).
    If parsing fails, parsed_obj is None and error_message contains info.
    """
    cleaned = strip_code_fence(s)
    try:
        return json.loads(cleaned), None
    except json.JSONDecodeError:
        # fallback: try extracting first {...} or [...] substring
        m = re.search(r"(\{(?:.|\s)*\}|\[(?:.|\s)*\])", cleaned)
        if m:
            try:
                return json.loads(m.group(1)), None
            except json.JSONDecodeError as e:
                return None, f"JSON decode failed for extracted substring: {e}"
        return None, "no JSON found or JSON invalid"

def normalize(parsed):
    """
    Normalize parsed JSON:
      - if list of one dict -> return that dict
      - if list of many -> return {"json_list": parsed}
      - otherwise return parsed as-is
    """
    if isinstance(parsed, list):
        if len(parsed) == 1 and isinstance(parsed[0], dict):
            return parsed[0]
        return {"json_list": parsed}
    return parsed

# keep one results array per prompt (saves one file per prompt level)
results_per_prompt = {pl: [] for pl in PROMPT_ORDER}

for i, image_path in enumerate(image_paths, start=1):
    print(f"\nProcessing {i}/{len(image_paths)}: {image_path}")

    # Parse location from filename; skip file if it doesn't match expected pattern
    try:
        continent, country_token, city, continent_dir = extract_location_from_path(image_path)
    except Exception as e:
        print(f" ⚠️ Skipping {image_path!r}: {e}")
        continue

    # Build explicit country list for this continent (with filename-aware replacement)
    try:
        countries, info = get_countries(continent, image_path)
        country_list = ", ".join(countries)
    except Exception as e:
        print(f" ⚠️ Could not list countries for {continent!r}: {e}")
        country_list = ""

    # Optional dataset hints
    parts = image_path.split(os.sep)
    dataset_continents = {"Africa","Asia","Europe","North America","South America","Oceania","Antarctica"}
    dataset_continent = next((p for p in parts if p in dataset_continents), None)
    dataset_country = None
    if dataset_continent:
        try:
            idx = parts.index(dataset_continent)
            dataset_country = parts[idx+1] if idx+1 < len(parts) else None
        except ValueError:
            pass
    dataset_split_hint = next((p for p in parts if p.upper() in {"TOP","MIDDLE","LAST"}), None)

    # 4) Run all L3 prompts for this image
    for pl in PROMPT_ORDER:
        PROMPT_LEVEL = pl
        prompt_text = PROMPTS[pl].format(
            given_continent=continent,
            country_list=country_list
        )
        print(f"\n  ➤ [{pl}] prompt:\n{prompt_text}\n")

        warn = None
        try:
            raw_output = infer_img(image_path, prompt_text)
            print(f"  ✔ {pl}: infer_img() full-res ok")
        except RuntimeError as e:
            msg = str(e).lower()
            if "cuda out of memory" in msg:
                torch.cuda.empty_cache()
                print(f"  ⚠ {pl}: OOM on full-res. Resizing to 448 and retrying…")
                try:
                    img = Image.open(image_path).convert("RGB").resize((448, 448))
                    with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmp:
                        tmp_path = tmp.name
                        img.save(tmp_path, format="JPEG")
                    raw_output = infer_img(tmp_path, prompt_text)
                    print(f"  ✔ {pl}: infer_img() resized ok")
                    warn = "OOM_resized"
                except Exception as e2:
                    print(f"  ❌ {pl}: failed after resize: {e2}")
                    raw_output = None
                    warn = "no_output"
            else:
                print(f"  ❌ {pl}: runtime error: {e}")
                raw_output = None
                warn = "no_output"
        except Exception as e:
            print(f"  ❌ {pl}: unexpected error: {e}")
            raw_output = None
            warn = "no_output"

        # 5) Normalize output into a single value
        if raw_output is None:
            output_value = None
            warn = warn or "no_output"
        elif isinstance(raw_output, (dict, list)):
            output_value = normalize(raw_output)
        else:
            parsed, perr = try_parse_json_from_string(raw_output)
            if parsed is not None:
                output_value = normalize(parsed)
            else:
                output_value = raw_output
                if not warn:
                    warn = f"json_parse_failed: {perr}" if perr else "json_parse_failed"

        # 6) Record + save (per prompt level file)
        rec = {
            "image_path":         image_path,
            "model":              MODEL_NAME,
            "variant":            VARIANT,
            "task":               TASK,
            "task_level":         TASK_LEVEL,
            "prompt_set":         "RQ1",
            "prompt_level":       PROMPT_LEVEL,
            "run_count":          RUN_COUNT,
            "prompt":             prompt_text,
            "output":             output_value,

            "dataset_continent":  dataset_continent,
            "dataset_country":    dataset_country,
            "dataset_split_hint": dataset_split_hint,

            "temperature":        GEN_TEMPERATURE,
            "max_new_tokens":     GEN_MAX_NEW_TOKENS,
            "do_sample":          GEN_DO_SAMPLE,
            "model_repo":         "OpenGVLab/InternVL3-8B",
            "image_size":         IMAGE_SIZE,
            "device":             ("cuda" if any(p.is_cuda for p in model.parameters()) else "cpu"),
            "dtype":              str(next(model.parameters()).dtype).replace("torch.", ""),
            "warning":            warn
        }
        results_per_prompt[pl].append(rec)

        # persist frequently (one file per prompt level)
        save_results(
            results_per_prompt[pl],
            MODEL_NAME, VARIANT, TASK,
            TASK_LEVEL,
            pl,
            RUN_COUNT,
            output_dir=OUTPUT_DIR
        )

    torch.cuda.empty_cache()
    print("\n------------------------------------------------------")

print("✅ Done.")


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.



Processing 1/360: /content/drive/MyDrive/RQ1_Experiments/RQ1_Data_Batch1/Africa/Benin/Africa_Benin_B1_MIDDLE.png

  ➤ [prompt1] prompt:
Assume the image is from Africa. Analyze it and answer two things:

1) Which country is this most likely from? (choose from Nigeria, South Africa, Egypt, Seychelles, Comoros, Benin, Mozambique, Rwanda, Botswana)

2) What is the most suitable label: political expression, vandalism, decoration, cultural identity, or advertisement?

  ✔ prompt1: infer_img() full-res ok


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


 Saved results to /content/drive/MyDrive/RQ1_Experiments/results/InternVL3-8B/RQ1_InternVL3-8B_8B_country_L3_prompt1_r1.json

  ➤ [prompt2] prompt:
You are a cultural anthropologist viewing street visuals. Given that mural in image is located in Africa, give these four pieces of info:

- COUNTRY (choose exactly one: Nigeria, South Africa, Egypt, Seychelles, Comoros, Benin, Mozambique, Rwanda, Botswana)
- LABEL (choose exactly one: political expression / vandalism / decoration / cultural identity / advertisement)
- CONFIDENCE (0–100%)
- TOP CUE (2–3 sentences describing the key visual/cultural cue)

Be concise and calibrate CONFIDENCE to how strongly the visual cues support your claims.

  ✔ prompt2: infer_img() full-res ok


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


  ✔ prompt2: infer_img() full-res ok
 Saved results to /content/drive/MyDrive/RQ1_Experiments/results/InternVL3-8B/RQ1_InternVL3-8B_8B_country_L3_prompt2_r1.json

  ➤ [prompt3] prompt:
Moderate a brief debate among a Geographer, a Street Artist, and a Museum Curator about this image, assuming it originates from Asia. For each expert, give:

- ROLE: <role name>
- COUNTRY (choose exactly one: China, India, Indonesia, United Arab Emirates, Singapore, Kazakhstan, Timor-Leste, Maldives, Brunei)
- LABEL (political expression / vandalism / decoration / cultural identity / advertisement)
- SHORT_RATIONALE (one concise sentence)

After the three positions, write a 2-paragraph moderator resolution:
1) Compare the three rationales and state which COUNTRY and LABEL you choose and why (trade-offs).
2) List 2 visual cues that were decisive and mention one alternative explanation you rejected.

Finish with: COUNTRY and LABEL (each on its own line).



Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


  ✔ prompt3: infer_img() full-res ok
 Saved results to /content/drive/MyDrive/RQ1_Experiments/results/InternVL3-8B/RQ1_InternVL3-8B_8B_country_L3_prompt3_r1.json

  ➤ [prompt4] prompt:
You are a cultural anthropologist & artist. Given that mural in image is located in Asia, provide:

- COUNTRY: two most likely countries (choose from: China, India, Indonesia, United Arab Emirates, Singapore, Kazakhstan, Timor-Leste, Maldives, Brunei) with percentages (sum ≈100), e.g. ""Iceland: 70%, Russia: 30%""
- WHY COUNTRY: 2–3 sentences explaining how visual cues suggest the chosen countries .

- LABEL: choose from (political expression, vandalism, decoration, cultural identity, advertisement) and also assign probabilities for all five labels (sum ≈100)
- WHY LABEL: 2–3 sentences linking cues to chosen label.

- TOP CUES: list three concrete cues that were decisive (1–6 words each)
- BIAS NOTE: 2–3 sentences reflecting how cultural bias might have affected the probability estimates.

Constraints:
-

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


### Download the zip file

In [ ]:
archive = "/content/results.tar.gz"
src_dir = "/content/drive/MyDrive/RQ1_Experiments/results"

!tar -czf "{archive}" -C "$(dirname "{src_dir}")" "$(basename "{src_dir}")"

from google.colab import files
files.download(archive)
